##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pandas DataFrame to Fairness Indicators Case Study


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/responsible_ai/fairness_indicators/tutorials/Fairness_Indicators_Pandas_Case_Study"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/fairness-indicators/blob/master/g3doc/tutorials/Fairness_Indicators_Pandas_Case_Study.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/fairness-indicators/tree/master/g3doc/tutorials/Fairness_Indicators_Pandas_Case_Study.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/fairness-indicators/g3doc/tutorials/Fairness_Indicators_Pandas_Case_Study.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Case Study Overview
In this case study we will apply [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started) and [Fairness Indicators](https://www.tensorflow.org/tfx/guide/fairness_indicators) to evaluate data stored as a Pandas DataFrame, where each row contains ground truth labels, various features, and a model prediction. We will show how this workflow can be used to spot potential fairness concerns, independent of the framework one used to construct and train the model. As in this case study, we can analyze the results from any machine learning framework (e.g. TensorFlow, JAX, etc) once they are converted to a Pandas DataFrame.
 
For this exercise, we will leverage the Deep Neural Network (DNN) model that was developed in the [Shape Constraints for Ethics with Tensorflow Lattice](https://colab.research.google.com/github/tensorflow/lattice/blob/master/docs/tutorials/shape_constraints_for_ethics.ipynb#scrollTo=uc0VwsT5nvQi) case study using the Law School Admissions dataset from the Law School Admissions Council (LSAC). This classifier attempts to predict whether or not a student will pass the bar, based on their Law School Admission Test (LSAT) score and undergraduate GPA.

## LSAC Dataset
The dataset used within this case study was originally collected for a study called '[LSAC National Longitudinal Bar Passage Study. LSAC Research Report Series](https://eric.ed.gov/?id=ED469370)' by Linda Wightman in 1998. The dataset is currently hosted [here](http://www.seaphe.org/databases.php).

*   **dnn_bar_pass_prediction**: The LSAT prediction from the DNN model.
*   **gender**: Gender of the student.
*   **lsat**: LSAT score received by the student.
*   **pass_bar**: Ground truth label indicating whether or not the student eventually passed the bar.
*   **race**: Race of the student.
*   **ugpa**: A student's undergraduate GPA.


In [2]:
!pip install -q -U pip==20.2

!pip install -q -U \
  tensorflow-model-analysis==0.39.0 \
  tensorflow-data-validation==1.8.0 \
  tfx-bsl==1.8.0

You should consider upgrading via the '/tmpfs/src/tf_docs_env/bin/python3.9 -m pip install --upgrade pip' command.


You should consider upgrading via the '/tmpfs/src/tf_docs_env/bin/python3.9 -m pip install --upgrade pip' command.


## Importing required packages:

In [3]:
import os
import tempfile
import pandas as pd
import six.moves.urllib as urllib
import pprint

import tensorflow_model_analysis as tfma
from google.protobuf import text_format

import tensorflow as tf
tf.compat.v1.enable_v2_behavior()

## Download the data and explore the initial dataset.

In [4]:
# Download the LSAT dataset and setup the required filepaths.
_DATA_ROOT = tempfile.mkdtemp(prefix='lsat-data')
_DATA_PATH = 'https://storage.googleapis.com/lawschool_dataset/bar_pass_prediction.csv'
_DATA_FILEPATH = os.path.join(_DATA_ROOT, 'bar_pass_prediction.csv')

data = urllib.request.urlopen(_DATA_PATH)

_LSAT_DF = pd.read_csv(data)

# To simpliy the case study, we will only use the columns that will be used for
# our model.
_COLUMN_NAMES = [
  'dnn_bar_pass_prediction',
  'gender',
  'lsat',
  'pass_bar',
  'race1',
  'ugpa',
]

_LSAT_DF.dropna()
_LSAT_DF['gender'] = _LSAT_DF['gender'].astype(str)
_LSAT_DF['race1'] = _LSAT_DF['race1'].astype(str)
_LSAT_DF = _LSAT_DF[_COLUMN_NAMES]

_LSAT_DF.head()

,dnn_bar_pass_prediction,gender,lsat,pass_bar,race1,ugpa
0,0.979804,female,44.0,1.0,white,3.5
1,0.979804,female,29.0,1.0,white,3.5
2,0.979804,male,36.0,1.0,white,3.5
3,0.979804,male,39.0,1.0,white,3.5
4,0.979804,male,48.0,1.0,white,3.5


## Configure Fairness Indicators.
There are several parameters that you’ll need to take into account when using Fairness Indicators with a DataFrame 

*   Your input DataFrame must contain a prediction column and label column from your model. By default Fairness Indicators will look for a prediction column called `prediction` and a label column called `label` within your DataFrame.
   *   If either of these values are not found a KeyError will be raised.

*   In addition to a DataFrame, you’ll also need to include an `eval_config` that should include the metrics to compute, slices to compute the metrics on, and the column names for example labels and predictions. 
   *   `metrics_specs` will set the metrics to compute. The `FairnessIndicators` metric will be required to render the fairness metrics and you can see a list of additional optional metrics [here](https://www.tensorflow.org/tfx/model_analysis/metrics).

   *   `slicing_specs` is an optional slicing parameter to specify what feature you’re interested in investigating. Within this case study race1 is used, however you can also set this value to another feature (for example gender in the context of this DataFrame). If `slicing_specs` is not provided all features will be included.
   *   If your DataFrame includes a label or prediction column that is different from the default `prediction` or `label`, you can configure the `label_key` and `prediction_key` to a new value.

*   If `output_path` is not specified a temporary directory will be created.

In [5]:
# Specify Fairness Indicators in eval_config.
eval_config = text_format.Parse("""
  model_specs {
    prediction_key: 'dnn_bar_pass_prediction',
    label_key: 'pass_bar'
  }
  metrics_specs {
    metrics {class_name: "AUC"}
    metrics {
      class_name: "FairnessIndicators"
      config: '{"thresholds": [0.50, 0.90]}'
    }
  }
  slicing_specs {
    feature_keys: 'race1'
  }
  slicing_specs {}
  """, tfma.EvalConfig())

# Run TensorFlow Model Analysis.
eval_result = tfma.analyze_raw_data(
  data=_LSAT_DF,
  eval_config=eval_config,
  output_path=_DATA_ROOT)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## Explore model performance with Fairness Indicators.

After running Fairness Indicators, we can visualize different metrics that we selected to analyze our models performance. Within this case study we’ve included Fairness Indicators and arbitrarily picked AUC.

When we first look at the overall AUC for each race slice we can see a slight discrepancy in model performance, but nothing that is arguably alarming.

*   **Asian**: 0.58
*   **Black**: 0.58
*   **Hispanic**: 0.58
*   **Other**: 0.64
*   **White**: 0.6

However, when we look at the false negative rates split by race, our model again incorrectly predicts the likelihood of a user passing the bar at different rates and, this time, does so by a lot. 

*   **Asian**: 0.01
*   **Black**: 0.05
*   **Hispanic**: 0.02
*   **Other**: 0.01
*   **White**: 0.01

Most notably the difference between Black and White students is about 380%, meaning that our model is nearly 4x more likely to incorrectly predict that a black student will not pass the bar, than a whilte student. If we were to continue with this effort, a practitioner could use these results as a signal that they should spend more time ensuring that their model works well for people from all backgrounds.

In [6]:
# Render Fairness Indicators.
tfma.addons.fairness.view.widget_view.render_fairness_indicator(eval_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'auc': {'dou…

# tfma.EvalResult

The [`eval_result`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult) object, rendered above in `render_fairness_indicator()`, has its own API that can be used to read TFMA results into your programs.

## [`get_slice_names()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult#get_slice_names) and [`get_metric_names()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult#get_metric_names)

To get the evaluated slices and metrics, you can use the respective functions.

In [7]:
pp = pprint.PrettyPrinter()

print("Slices:")
pp.pprint(eval_result.get_slice_names())
print("\nMetrics:")
pp.pprint(eval_result.get_metric_names())

Slices:
[(),
 (('race1', 'white'),),
 (('race1', 'hisp'),),
 (('race1', 'asian'),),
 (('race1', 'black'),),
 (('race1', 'nan'),),
 (('race1', 'other'),)]

Metrics:
['fairness_indicators_metrics/recall@0.9',
 'fairness_indicators_metrics/false_negative_rate@0.9',
 'auc',
 'fairness_indicators_metrics/negative_rate@0.9',
 'fairness_indicators_metrics/true_negative_rate@0.9',
 'fairness_indicators_metrics/true_positive_rate@0.5',
 'fairness_indicators_metrics/positive_rate@0.5',
 'fairness_indicators_metrics/true_negative_rate@0.5',
 'fairness_indicators_metrics/negative_rate@0.5',
 'fairness_indicators_metrics/false_omission_rate@0.9',
 'fairness_indicators_metrics/true_positive_rate@0.9',
 'fairness_indicators_metrics/precision@0.5',
 'fairness_indicators_metrics/false_discovery_rate@0.5',
 'fairness_indicators_metrics/positive_rate@0.9',
 'fairness_indicators_metrics/false_negative_rate@0.5',
 'fairness_indicators_metrics/false_positive_rate@0.5',
 'fairness_indicators_metrics/recall@0

## [`get_metrics_for_slice()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult#get_metrics_for_slice) and [`get_metrics_for_all_slices()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult#get_metrics_for_all_slices)

If you want to get the metrics for a particular slice, you can use `get_metrics_for_slice()`. It returns a dictionary mapping metric names to [metric values](https://github.com/tensorflow/model-analysis/blob/cdb6790dcd7a37c82afb493859b3ef4898963fee/tensorflow_model_analysis/proto/metrics_for_slice.proto#L194).

In [8]:
baseline_slice = ()
black_slice = (('race1', 'black'),)

print("Baseline metric values:")
pp.pprint(eval_result.get_metrics_for_slice(baseline_slice))
print("Black metric values:")
pp.pprint(eval_result.get_metrics_for_slice(black_slice))

Baseline metric values:
{'auc': {'doubleValue': 0.6286112880450339},
 'fairness_indicators_metrics/false_discovery_rate@0.5': {'doubleValue': 0.052173524948674464},
 'fairness_indicators_metrics/false_discovery_rate@0.9': {'doubleValue': 0.0502241746297722},
 'fairness_indicators_metrics/false_negative_rate@0.5': {'doubleValue': 4.7085412939071474e-05},
 'fairness_indicators_metrics/false_negative_rate@0.9': {'doubleValue': 0.012524719841793012},
 'fairness_indicators_metrics/false_omission_rate@0.5': {'doubleValue': 1.0},
 'fairness_indicators_metrics/false_omission_rate@0.9': {'doubleValue': 0.8159509202453987},
 'fairness_indicators_metrics/false_positive_rate@0.5': {'doubleValue': 1.0},
 'fairness_indicators_metrics/false_positive_rate@0.9': {'doubleValue': 0.9486740804106074},
 'fairness_indicators_metrics/negative_rate@0.5': {'doubleValue': 4.462891060829205e-05},
 'fairness_indicators_metrics/negative_rate@0.9': {'doubleValue': 0.014549024858303209},
 'fairness_indicators_metric

If you want to get the metrics for all slices, `get_metrics_for_all_slices()` returns a dictionary mapping each slice to the corresponding `get_metrics_for_slices(slice)`.

In [9]:
pp.pprint(eval_result.get_metrics_for_all_slices())

{(): {'auc': {'doubleValue': 0.6286112880450339},
      'fairness_indicators_metrics/false_discovery_rate@0.5': {'doubleValue': 0.052173524948674464},
      'fairness_indicators_metrics/false_discovery_rate@0.9': {'doubleValue': 0.0502241746297722},
      'fairness_indicators_metrics/false_negative_rate@0.5': {'doubleValue': 4.7085412939071474e-05},
      'fairness_indicators_metrics/false_negative_rate@0.9': {'doubleValue': 0.012524719841793012},
      'fairness_indicators_metrics/false_omission_rate@0.5': {'doubleValue': 1.0},
      'fairness_indicators_metrics/false_omission_rate@0.9': {'doubleValue': 0.8159509202453987},
      'fairness_indicators_metrics/false_positive_rate@0.5': {'doubleValue': 1.0},
      'fairness_indicators_metrics/false_positive_rate@0.9': {'doubleValue': 0.9486740804106074},
      'fairness_indicators_metrics/negative_rate@0.5': {'doubleValue': 4.462891060829205e-05},
      'fairness_indicators_metrics/negative_rate@0.9': {'doubleValue': 0.014549024858303209

## Conclusion
Within this case study we imported a dataset into a Pandas DataFrame that we then analyzed with Fairness Indicators. Understanding the results of your model and underlying data is an important step in ensuring your model doesn't reflect harmful bias. In the context of this case study we examined the the LSAC dataset and how predictions from this data could be impacted by a students race. The concept of “what is unfair and what is fair have been introduced in multiple disciplines for well over 50 years, including in education, hiring, and machine learning.”<sup>1</sup> Fairness Indicator is a tool to help mitigate fairness concerns in your machine learning model.

For more information on using Fairness Indicators and resources to learn more about fairness concerns see [here](https://www.tensorflow.org/responsible_ai/fairness_indicators/guide).

---

1. Hutchinson, B., Mitchell, M. (2018). 50 Years of Test (Un)fairness: Lessons for Machine Learning. https://arxiv.org/abs/1811.10104


## Appendix

Below are a few functions to help convert ML models to Pandas DataFrame.


In [10]:
# TensorFlow Estimator to Pandas DataFrame:

# _X_VALUE =  # X value of binary estimator.
# _Y_VALUE =  # Y value of binary estimator.
# _GROUND_TRUTH_LABEL =  # Ground truth value of binary estimator.

def _get_predicted_probabilities(estimator, input_df, get_input_fn):
  predictions = estimator.predict(
      input_fn=get_input_fn(input_df=input_df, num_epochs=1))
  return [prediction['probabilities'][1] for prediction in predictions]

def _get_input_fn_law(input_df, num_epochs, batch_size=None):
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
      x=input_df[[_X_VALUE, _Y_VALUE]],
      y=input_df[_GROUND_TRUTH_LABEL],
      num_epochs=num_epochs,
      batch_size=batch_size or len(input_df),
      shuffle=False)

def estimator_to_dataframe(estimator, input_df, num_keypoints=20):
  x = np.linspace(min(input_df[_X_VALUE]), max(input_df[_X_VALUE]), num_keypoints)
  y = np.linspace(min(input_df[_Y_VALUE]), max(input_df[_Y_VALUE]), num_keypoints)

  x_grid, y_grid = np.meshgrid(x, y)

  positions = np.vstack([x_grid.ravel(), y_grid.ravel()])
  plot_df = pd.DataFrame(positions.T, columns=[_X_VALUE, _Y_VALUE])
  plot_df[_GROUND_TRUTH_LABEL] = np.ones(len(plot_df))
  predictions = _get_predicted_probabilities(
      estimator=estimator, input_df=plot_df, get_input_fn=_get_input_fn_law)
  return pd.DataFrame(
      data=np.array(np.reshape(predictions, x_grid.shape)).flatten())